# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [4]:
# Write your code below.
%load_ext dotenv
%dotenv ../../01_materials/labs/.env


In [8]:
import dask.dataframe as dd

c:\Users\syednamr\AppData\Local\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 10.0.1. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [28]:
import os
import glob

price_data_dir = os.getenv('PRICE_DATA')
parquet_files = glob.glob(os.path.join(price_data_dir, '**', '*.parquet'), recursive=True)


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [26]:
# Write your code below.
dd_feat_list = []

for file in parquet_files:
    dd_px = dd.read_parquet(file, engine="pyarrow")

    dd_px['Close_lag'] = dd_px.groupby('ticker')['Close'].shift(1)
    dd_px['Adj_Close_lag'] = dd_px.groupby('ticker')['Adj Close'].shift(1)
    dd_px['returns'] = (dd_px['Adj Close'] / dd_px['Adj_Close_lag']) - 1
    dd_px['hi_lo_range'] = dd_px['High'] - dd_px['Low']
    
    dd_feat_list.append(dd_px)

dd_feat = dd.concat(dd_feat_list)
dd_feat = dd_feat.compute()

print(dd_feat.head())

             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
A      2000-01-03  56.330471  56.464592  48.193848  51.502148  43.463024   
A      2000-01-04  48.730328  49.266811  46.316166  47.567955  40.142937   
A      2000-01-05  47.389126  47.567955  43.141991  44.617310  37.652866   
A      2000-01-06  44.080830  44.349072  41.577251  42.918453  36.219173   
A      2000-01-07  42.247852  47.165592  42.203148  46.494991  39.237438   

         Volume       sector                       subsector  year  Close_lag  \
ticker                                                                          
A       4674353  Health Care  Life Sciences Tools & Services  2000        NaN   
A       4765083  Health Care  Life Sciences Tools & Services  2000  51.502148   
A       5758642  Health Care  Life Sciences Tools & Services  2000  47.567955   
A       2534434  Health Care  Life Sciences Tools & Services  

C:\Users\syednamr\AppData\Local\Temp\ipykernel_5864\2042698438.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_px['Close_lag'] = dd_px.groupby('ticker')['Close'].shift(1)
C:\Users\syednamr\AppData\Local\Temp\ipykernel_5864\2042698438.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_px['Adj_Close_lag'] = dd_px.groupby('ticker')['Adj Close'].shift(1)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [27]:
# Write your code below.
dd_feat['rolling_return'] = dd_feat.groupby('ticker')['returns'].rolling(10, min_periods=1).mean().reset_index(level=0, drop=True)
print(dd_feat.head())

             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
A      2000-01-03  56.330471  56.464592  48.193848  51.502148  43.463024   
A      2000-01-04  48.730328  49.266811  46.316166  47.567955  40.142937   
A      2000-01-05  47.389126  47.567955  43.141991  44.617310  37.652866   
A      2000-01-06  44.080830  44.349072  41.577251  42.918453  36.219173   
A      2000-01-07  42.247852  47.165592  42.203148  46.494991  39.237438   

         Volume       sector                       subsector  year  Close_lag  \
ticker                                                                          
A       4674353  Health Care  Life Sciences Tools & Services  2000        NaN   
A       4765083  Health Care  Life Sciences Tools & Services  2000  51.502148   
A       5758642  Health Care  Life Sciences Tools & Services  2000  47.567955   
A       2534434  Health Care  Life Sciences Tools & Services  

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

It is possible to accomplish the moving average return in both Pandas and Dask, but Pandas is a bit simpler to use than Dask, and it is easier to produce quicker solutions using Pandas. Unfortunately, Pandas is limited by memory constraints, so it is not always possible to use it when the data is too large in volume.

It would not have been better to do it in Dask, because Pandas is simpler to use; however, given the memory limits, it wouldn't be possible.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.